In [4]:
import pandas as pd

import segmentation as s

In [6]:
data = pd.read_csv('data/preprocessed_data.csv', 
                   index_col = "customer_id")
data
# temp_data = data[s.temporal_data]
# temp_data

,customer_region,customer_age,vendor_count,product_count,is_chain,first_order,last_order,last_promo,payment_method,CUI_American,...,avg_order_hour,total_spend,avg_spend_prod,is_repeat_customer,weekend_orders,weekday_orders,0_7h,8_14h,15_19h,20_23h
customer_id,,,,,,,,,,,,,,,,,,,,,
1b8f824d5e,2360,18.0,2,5,1,0,1,DELIVERY,DIGI,0.00,...,18.0,28.88,5.776,1,2,0,0,0,2,0
5d272b9dcb,8670,17.0,2,2,2,0,1,DISCOUNT,DIGI,12.82,...,10.5,19.21,9.605,1,2,0,0,2,0,0
f6d1b2ba63,4660,38.0,1,2,2,0,1,DISCOUNT,CASH,9.20,...,10.0,9.20,4.600,1,2,0,0,2,0,0
180c632ed8,4660,26.0,2,3,1,0,2,DELIVERY,DIGI,0.00,...,12.5,31.56,10.520,1,1,1,0,2,0,0
4eb37a6705,4660,20.0,2,5,0,0,2,NO DISCOUNT,DIGI,14.57,...,8.5,55.44,11.088,1,1,1,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f4e366c281,8670,30.0,1,1,1,0,0,FREEBIE,DIGI,0.00,...,13.0,18.04,18.040,0,1,0,0,1,0,0
f6b6709018,8670,26.0,1,1,0,0,0,DELIVERY,DIGI,18.04,...,16.0,18.04,18.040,0,1,0,0,0,1,0
f74ad8ce3f,8670,24.0,1,1,1,0,0,NO DISCOUNT,DIGI,0.00,...,5.0,17.79,17.790,0,1,0,1,0,0,0
